In [11]:
import pandas as pd
import plotly as px
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.backends.backend_pdf import PdfPages

In [12]:
def pattern_comportamento(df,perc_val):
    #Intervalli sul grafico 770sec, media valori ogni 30sec
    df_max =df.resample('30S').mean()
    max_value = df_max['consumption'].max()

    #if max_value==0:return "0"#0 emissioni tutto il giorno

    #ELIMINO TUTTE LE EMISSIONI INFERIORI A UNA CERTA SOGLIA
    und_value=(max_value/100)*perc_val
    for index, row in df.iterrows():
        if row['consumption'] <= und_value:
            df.at[index, 'consumption'] = 0

    df_inter =df.resample('770S').mean() #abbiamo solo emissioni 'importanti'
    
    #VEDE GLI INTERVALLI E DURATA
    count_inter = 0
    in_interval = False
    start_date = None
    end_date = None
    duration=0
    intervals = []
    for date, consumption in df_inter['consumption'].items():
        if consumption > 0:#abbiamo delle emissioni
            if not in_interval:#inizio intervallo
                in_interval = True
                start_date = date
                duration = 1 
                count_inter += 1
            else:#continuo intervallo
                duration += 1
        else:#no emissioni
            if in_interval:
                in_interval = False
                end_date=date
                intervals.append((start_date, end_date, duration))    
    if in_interval:
        end_date=date
        intervals.append((start_date, end_date, duration))
    print('DEBUG ARRAY INTERVALLI\n')
    print(intervals)

    #un intervallo vale 770s=12,8m
    #CREAZIONE PATTERN
    count_intervalli=len(intervals)
    pattern=""
    #if count_intervalli>3: pattern += "5"#MOLTE EMISSIONI
    #else: pattern += str(count_intervalli) #1,2,3 

    pattern += str(count_intervalli)

    #AGGIUNTA SULL'ANANLISI PER MENO DI 4 INTERVALLI
    if count_intervalli<=3:
        for date_inizio,date_fine, durata in intervals:
            if durata>3: pattern += "L"#INTERVALLO LUNGO
            elif durata>1:  pattern += "M"#INTERVALLO MEDIO
            else: pattern += "P"#INTERVALLO PICCOLO     

    pattern += "_"   
    
    #CREAZIONE ARRAY CON DATA INIZIO E FINE PER FUTURI CALCOLI
    diff_tra_intervalli = []
    dates = [(interval[0]).to_pydatetime() for interval in intervals]#array inizio intervalli
    dates_fine = [(interval[1]).to_pydatetime() for interval in intervals]#array fine intervalli
    
    #CALCOLA TIPI DI INTERVALLI TRA I MOMENTI DI USO DEL DISPOSITIVO
    for i in range(len(dates_fine) - 1):
        difference  = (dates[i+1] - dates_fine[i]).total_seconds()
        diff=difference/770
        diff_tra_intervalli.append(diff)
    print(f'\nspazio tra gli intervalli: {diff_tra_intervalli}\n')
    count_l=0
    count_m=0
    count_p=0
    for x in diff_tra_intervalli:#calcolo dei tipi di intervalli
        if x>3: count_l += 1#intervallo di tempo LUNGO
        elif x>1:  count_m += 1#intervallo di tempo MEDIO
        else: count_p += 1#intervallo di tempo PICCOLO

    #ANALISI GENERALE INTERVALLI
    if count_l==len(dates): pattern+="S0"#MOLTO MOLTO SPARSO
    elif count_l>=(3/4)*len(dates):#MOLTO SPARSO
        if count_m>=(3/4)*count_p: pattern+="S1a"
        elif count_m>=(1/2)*count_p: pattern+="S1b"
        elif count_m>=(1/4)*count_p: pattern+="S1c"
        elif count_m==0: pattern+="S1d"
    elif count_l>=(1/2)*len(dates):#SPARSO
        if count_m>=(3/4)*count_p: pattern+="S2a"
        elif count_m>=(1/2)*count_p: pattern+="S2b"
        elif count_m>=(1/4)*count_p: pattern+="S2c"
        elif count_m==0: pattern+="S2d"
    elif count_l>=(1/4)*len(dates):#MEDIAMENTE SPARSO
        if count_m>=(3/4)*count_p: pattern+="M1a"
        elif count_m>=(1/2)*count_p: pattern+="M1b"
        elif count_m>=(1/4)*count_p: pattern+="M1c"
        elif count_m==0: pattern+="M1d"
    elif count_l==0:#NON MOLTO SPARSO
        if count_m>=(3/4)*count_p: pattern+="M2a"
        elif count_m>=(1/2)*count_p: pattern+="M2b"
        elif count_m>=(1/4)*count_p: pattern+="M2c"
        elif count_m==0: pattern+="P1" #CONCENTRATO
    return pattern

    #logica
    #if x>=(3/4)*y:
    #elif x>=(1/2)*y:
    #elif x>=(1/4)*y:
    #elif x==0:

In [11]:
def range_plot_media(df,date,perc_val):
    #Media delle emissioni degli intervalli di X secondi
    df=df.resample('30S').mean()
    max_value = df['consumption'].max()

    #if max_value==0:return
    #ELIMINO TUTTE LE EMISSIONI INFERIORI A UNA CERTA SOGLIA
    und_value=(max_value/100)*perc_val
    for index, row in df.iterrows():
        if row['consumption'] <= und_value:
            df.at[index, 'consumption'] = 0

    plt.figure(figsize=(10, 6))

    #Grafico per tutte le emissioni
    plt.plot(df.index, df['consumption'], label={'consumption': 'VolumeMedia [L]'}, color='red', linewidth=0.5)

    #Formatta le date sull'asse x
    plt.gca().xaxis.set_major_locator(mdates.SecondLocator(interval=770))

    #Tick per i giorni con colore differenti
    plt.gca().xaxis.set_minor_formatter(mdates.DateFormatter('%m-%d'))
    plt.gca().xaxis.set_minor_locator(mdates.DayLocator(interval=1))
    plt.tick_params(axis='x', which='minor', colors='red')

    # Impostazioni dell'asse y con tick ogni 100 unità
    plt.gca().yaxis.set_major_locator(plt.MultipleLocator(100))
    max_value = df['consumption'].max()
    if max_value>1000:
        plt.gca().yaxis.set_minor_locator(plt.MultipleLocator(50))
    else:
        plt.gca().yaxis.set_minor_locator(plt.MultipleLocator(20))

    #Impostazioni della griglia
    plt.xlabel('Date')
    plt.ylabel('Consumption (L)')
    plt.title(f'Emissioni del giorno {date}')
    plt.legend()
    plt.grid(True, which='major', linestyle='-', linewidth=1)
    plt.grid(True, which='minor', linestyle='-', linewidth=5)
    plt.grid(True, which='major', axis='y', linestyle='-', linewidth=1)
    plt.grid(True, which='minor', axis='y', linestyle='-', linewidth=1)
    plt.xticks(rotation=90)
    plt.show() #X CONTROLLI GRAFIIC

    # Chiudere il grafico per liberare memoria
    plt.close()

In [10]:
def pattern_comportamento_test(df,day_emiss,mm,mt,ms):
    #Intervalli sul grafico 770sec=12,8m, media valori ogni 30sec

    df_inter =df.resample('770S').mean() #abbiamo solo emissioni 'importanti'
    
    #VEDE GLI INTERVALLI E DURATA
    in_interval = False
    start_date = None
    end_date = None
    duration=0
    intervals = []
    for date, consumption in df_inter['consumption'].items():
        if consumption > 0:#abbiamo delle emissioni
            if not in_interval:#inizio intervallo
                in_interval = True
                start_date = date
                duration = 1 
            else:#continuo intervallo
                duration += 1
        else:#no emissioni
            if in_interval:
                in_interval = False
                end_date=date
                intervals.append((start_date, end_date, duration))    
    if in_interval:
        end_date=date
        intervals.append((start_date, end_date, duration))
    #DEBUG print(intervals)

    #CREAZIONE PATTERN
    #ANALISI EMISSIONI 
    print(f'{day_emiss }')
    #mm
    pattern=""
    if day_emiss<(mm/4):
        pattern+= "CA"
    elif day_emiss<(mm/2):
        pattern+= "CB"
    elif day_emiss<(mm*(3/4)):
        pattern+= "CC"
    elif day_emiss<mm:
        pattern+= "CD"
    elif day_emiss<(mm*(5/4)):
        pattern+= "CE"
    elif day_emiss<(mm*(3/2)):
        pattern+= "CF"
    elif day_emiss<(mm*(7/4)):
        pattern+= "CG"
    else:
        pattern+= "CH"
        
    #mt
    if day_emiss<(mt/4):
        pattern+= "MA"
    elif day_emiss<(mt/2):
        pattern+= "MB"
    elif day_emiss<(mt*(3/4)):
        pattern+= "MC"
    elif day_emiss<mt:
        pattern+= "MD"
    elif day_emiss<(mt*(5/4)):
        pattern+= "ME"
    elif day_emiss<(mt*(3/2)):
        pattern+= "MF"
    elif day_emiss<(mt*(7/4)):
        pattern+= "MG"
    else:
        pattern+= "MH"

    #ms
    if day_emiss<(ms / 4):
        pattern+= "LA"
    elif day_emiss<(ms / 2):
        pattern+= "LB"
    elif day_emiss<(ms * (3 / 4)):
        pattern+= "LC"
    elif day_emiss<ms:
        pattern+= "LD"
    elif day_emiss<(ms * (5 / 4)):
        pattern+= "LE"
    elif day_emiss<(ms * (3 / 2)):
        pattern+= "LF"
    elif day_emiss<(ms * (7 / 4)):
        pattern+= "LG"
    else:
        pattern+= "LH"
    
    #ANALISI INTERVALLI
    count_intervalli=len(intervals)
    pattern+= str(count_intervalli) #NUMERO DI INTERVALLI 

    #AGGIUNTA SULL'ANANLISI PER MENO DI 4 INTERVALLI
    if count_intervalli<4:#osservazione potrei farlo per tutti o allargare il numero 
        for _,_, durata in intervals:
            if durata>3: pattern += "L"#INTERVALLO LUNGO
            elif durata>1:  pattern += "M"#INTERVALLO MEDIO
            else: pattern += "P"#INTERVALLO PICCOLO     

    pattern += "_"   
    
    #CHANGE
    count_l=0
    count_m=0
    count_p=0
    for i in range(len(intervals) - 1):
        ris=int(( (intervals[i+1][0]).to_pydatetime() - (intervals[i][1]).to_pydatetime() ).total_seconds()/770)
        if ris>3: count_l += 1#intervallo di tempo LUNGO
        elif ris>1:  count_m += 1#intervallo di tempo MEDIO
        else: count_p += 1#intervallo di tempo PICCOLO
        #diff_tra_intervalli.append(  ris )

    tot=count_l+(count_m+count_p)
    #ANALISI GENERALE INTERVALLI
    if count_l==tot: pattern+="S0"#MOLTO MOLTO SPARSO
    elif count_l>=(3/4)*tot:#MOLTO SPARSO
        if count_m==(count_m+count_p): pattern+="S1a"
        elif count_m>=(3/4)*(count_m+count_p): pattern+="S1b"
        elif count_m>=(1/2)*(count_m+count_p): pattern+="S1c"
        elif count_m>=(1/4)*(count_m+count_p): pattern+="S1d"
        elif count_m>0: pattern+="S1e"
        elif count_m==0: pattern+="S1f"
    elif count_l>=(1/2)*tot:#SPARSO
        if count_m==(count_m+count_p): pattern+="S2a"
        elif count_m>=(3/4)*(count_m+count_p): pattern+="S2b"
        elif count_m>=(1/2)*(count_m+count_p): pattern+="S2c"
        elif count_m>=(1/4)*(count_m+count_p): pattern+="S2d"
        elif count_m>0: pattern+="S2e"
        elif count_m==0: pattern+="S2f"
    elif count_l>=(1/4)*tot:#MEDIAMENTE SPARSO
        if count_m==(count_m+count_p): pattern+="M1a"
        elif count_m>=(3/4)*(count_m+count_p): pattern+="M1b"
        elif count_m>=(1/2)*(count_m+count_p): pattern+="M1c"
        elif count_m>=(1/4)*(count_m+count_p): pattern+="M1d"
        elif count_m>0: pattern+="M1e"
        elif count_m==0: pattern+="M1f"
    elif count_l>0:#NON MOLTO SPARSO
        if count_m==(count_m+count_p): pattern+="M2a"
        elif count_m>=(3/4)*(count_m+count_p): pattern+="M2b"
        elif count_m>=(1/2)*(count_m+count_p): pattern+="M2c"
        elif count_m>=(1/4)*(count_m+count_p): pattern+="M2d"
        elif count_m>0: pattern+="M2e"
        elif count_m==0: pattern+="P0" #CONCENTRATO
    elif count_l==0:#NON MOLTO SPARSO
        if count_m==(count_m+count_p): pattern+="P1a"
        elif count_m>=(3/4)*(count_m+count_p): pattern+="P1b"
        elif count_m>=(1/2)*(count_m+count_p): pattern+="P1c"
        elif count_m>=(1/4)*(count_m+count_p): pattern+="P1d"
        elif count_m>0: pattern+="P1e"
        elif count_m==0: pattern+="P1f" #CONCENTRATO
    print("DEBUG")
    return pattern

In [10]:
def calcola_media_mensile(df):
    periodi_mensili = [
        (pd.Timestamp('2019-01-01'), pd.Timestamp('2019-01-31')),
        (pd.Timestamp('2019-02-01'), pd.Timestamp('2019-02-28')),
        (pd.Timestamp('2019-03-01'), pd.Timestamp('2019-03-31')),
        (pd.Timestamp('2019-04-01'), pd.Timestamp('2019-04-30')),
        (pd.Timestamp('2019-05-01'), pd.Timestamp('2019-05-31')),
        (pd.Timestamp('2019-06-01'), pd.Timestamp('2019-06-30')),
        (pd.Timestamp('2019-07-01'), pd.Timestamp('2019-07-31')),
        (pd.Timestamp('2019-08-01'), pd.Timestamp('2019-08-31')),
        (pd.Timestamp('2019-09-01'), pd.Timestamp('2019-09-30')),
        (pd.Timestamp('2019-10-01'), pd.Timestamp('2019-10-31')),
        (pd.Timestamp('2019-11-01'), pd.Timestamp('2019-11-30')),
        (pd.Timestamp('2019-12-01'), pd.Timestamp('2019-12-31'))
    ]

    # Calcolare la media per ciascun periodo
    media_mensile = []
    for start, end in periodi_mensili:
        media = df.loc[start:end].mean()
        media_mensile.append(media.values[0])  # Assumendo che ci sia una sola colonna di valori

    return media_mensile

In [9]:
def calcola_media_trimestrale(df):
    periodi_trimestrali = [
        (pd.Timestamp('2019-01-01'), pd.Timestamp('2019-03-31')),
        (pd.Timestamp('2019-04-01'), pd.Timestamp('2019-06-30')),
        (pd.Timestamp('2019-07-01'), pd.Timestamp('2019-09-30')),
        (pd.Timestamp('2019-10-01'), pd.Timestamp('2019-12-31'))
    ]

    # Calcolare la media per ciascun periodo
    media_trimestrale = []
    for start, end in periodi_trimestrali:
        media = df.loc[start:end].mean()
        media_trimestrale.append(media.values[0])  # Assumendo che ci sia una sola colonna di valori

    return media_trimestrale

In [8]:
def calcola_media_semestrale(df_final):
    periodi_semestrali = [
        (pd.Timestamp('2019-01-01'), pd.Timestamp('2019-06-30')),
        (pd.Timestamp('2019-07-01'), pd.Timestamp('2019-12-31'))
    ]
    media_semestrale = []
    for start, end in periodi_semestrali:
        media = df_final.loc[start:end].mean()
        media_semestrale.append(media.values[0])  # Assumendo che ci sia una sola colonna di valori

    return media_semestrale

In [7]:
def calcola_media_annuale(df):
    return df['consumption'].mean()

In [6]:
def read_fixture_sec(fixture: str, rule='S'):
    # Legge il CSV
    df = pd.read_csv(f'./data/feed_{fixture}.MYD.csv', delimiter=' ', header=None, names=['date', fixture])
    df['date'] = pd.to_datetime(df['date'], unit='s')
    
    # Rinomina la colonna fixture in 'consumption'
    df.rename(columns={fixture: 'consumption'}, inplace=True)
    df.set_index('date', inplace=True)
    
    # Resample il DataFrame usando la frequenza specificata (secondi 'S')
    df_secondly = df.resample(rule).sum()
    
    # Creare un DataFrame vuoto con il range di date specificato a livello di secondi
    date_rng = pd.date_range(start='2019-01-01', end='2019-12-31 23:59:59', freq='S')
    df_final = pd.DataFrame(date_rng, columns=['date'])
    df_final.set_index('date', inplace=True)
    
    # Unire il DataFrame secondario con il DataFrame vuoto per assicurarsi che tutte le date siano presenti
    df_final = df_final.join(df_secondly, how='left').fillna(0)
    
    return df_final

In [5]:
def read_fixture(fixture: str, rule='T'):
    #Legge CSV
    df = pd.read_csv(f'./data/feed_{fixture}.MYD.csv', delimiter=' ', header=None, names=['date', fixture])
    df['date'] = pd.to_datetime(df['date'], unit='s')
    
    # Rinomina la colonna fixture in 'consumption'
    df.rename(columns={fixture: 'consumption'}, inplace=True)
    df.set_index('date', inplace=True)
    
    # Resample il DataFrame usando la frequenza specificata (minutale 'T')
    df_minutely = df.resample(rule).sum()
    
    # Ora raggruppiamo i dati minutali in giornate
    df_daily = df_minutely.resample('D').sum()

    # Creare un DataFrame vuoto con il range di date specificato
    date_rng = pd.date_range(start='2019-01-01', end='2019-12-31' , freq='D')
    df_final = pd.DataFrame(date_rng, columns=['date'])
    df_final.set_index('date', inplace=True)
    
    # Unire il DataFrame giornaliero con il DataFrame vuoto per assicurarsi che tutte le date siano presenti
    df_final = df_final.join(df_daily, how='left').fillna(0)
    
    return df_final

In [ ]:
from collections import Counter
#per vedere le differenze basta che cambi perc_val o eliminare la parte di codice tra gli asterisci

df_dispositivo_sec=read_fixture_sec('Washbasin')
df_dispositivo_emissioni=read_fixture('Washbasin')
diz_patterns = {}
dizionario_pattern = {}
perc_val=5 #CAMBIARE IL VALORE PER AUMENTARE/DIMINUIRE IL MINIMO ACCETTABILE PER OGNI INTERVALLO

start_date = pd.Timestamp('2019-01-07')#PRIMO LUNEDI 
end_date = pd.Timestamp('2019-12-29')#ULTIMA DOMENICA 
date_rng = pd.date_range(start= start_date, end=end_date, freq='D')

#CALCOLI DELLA MEDIA MENSILE, TRIMESTRALE, SEMESTRALE
mm_array=calcola_media_mensile(df_dispositivo_emissioni)
mt_array=calcola_media_trimestrale(df_dispositivo_emissioni)
ms_array=calcola_media_semestrale(df_dispositivo_emissioni)
ma=calcola_media_annuale(df_dispositivo_emissioni)
#DEBUG 
#print(f'Media MENSILE: {mm_array}\n')
#print(f'Media TRIMESTRALE: {mt_array}\n')
#print(f'Media SEMESTRALE: {ms_array}\n')
#print(f'Media ANNUALE: {ma}\n')

for date in date_rng:
    #print(f'{date}\n')
    start_date = pd.Timestamp(date)
    end_date = pd.Timestamp(date).replace(hour=23, minute=59, second=59)
    patterns=""

    #CAPISCE ALGORTIMO
    df= df_dispositivo_sec[start_date:end_date]
    df_dispositivo_emis=df_dispositivo_emissioni[start_date:end_date] #calcoli su emissioni

    #***********CONTROLLA SE SUPERIAMO LO 0.1% DELLA MEDIA ANNUALE DI EMISSIONI NELLA GIORNATA 
    day_emiss=df_dispositivo_emis['consumption'].to_numpy()
    day_emiss=day_emiss[0]
    temp=(ma/100)*0.1
    if day_emiss<temp: patterns="0"
    else:
    #**********
        df_max =df.resample('30S').mean()
        max_value = df_max['consumption'].max()
        und_value=(max_value/100)*perc_val
        #ELIMINO TUTTE LE EMISSIONI INFERIORI A UNA CERTA SOGLIA
        for index, row in df.iterrows():
            if row['consumption'] < und_value:
                df.at[index, 'consumption'] = 0
        
        if max_value!=0:
            #GRAFICI range_plot_media(df,start_date,perc_val)
            
            #VALORI MEDIE IN BASE AL GIORNO
            num_mese=date.month
            #mensile
            mm=mm_array[num_mese-1]
            #trimestrale
            if num_mese<=3: mt=mt_array[0]
            elif num_mese<=6: mt=mt_array[1]
            elif num_mese<=9: mt=mt_array[2]
            else: mt=mt_array[3]
            #semestrale
            if num_mese<=6: ms=ms_array[0]
            else: ms=ms_array[1]

            patterns=pattern_comportamento_test(df,day_emiss,mm,mt,ms)
        else: patterns="0"  
    #DEBUG print(f'Pattern grafico {patterns}\n')

    #CODICE PER OTTENERE LE VOLTE CHE UN PATTERN SI PRESENTA
    #Calcola presenze dei pattern
    if patterns in diz_patterns:#se lo trova 
        dizionario_pattern[patterns].append(df_max)
        diz_patterns[patterns] += 1
    else:
        dizionario_pattern[patterns]=[]
        dizionario_pattern[patterns].append(df_max)
        diz_patterns[patterns] = 1
    #CARTELLA DOVE SALVARE I GRAFICI patterngrafico
print(f'Numero di chiavi nel dizionario: {len(diz_patterns)}:{len(dizionario_pattern)}')
print(diz_patterns)
for key,value in dizionario_pattern.items():
        print(f'\n{key}')
        pdf_filename = f'patterngrafico\{key}.pdf'
        with PdfPages(pdf_filename) as pdf:
            for graf in value:
                # Stampa l'oggetto Figure 
                if not isinstance(graf, pd.DataFrame) or 'consumption' not in graf.columns:#CONTROLLO 
                    print(f'Warning: Invalid DataFrame for key {key}')
                    continue

                #Media delle emissioni degli intervalli di X secondi
                df=graf.resample('30S').mean()

                plt.figure(figsize=(10, 6))

                #Grafico per tutte le emissioni
                plt.plot(df.index, df['consumption'], label={'consumption': 'VolumeMedia [L]'}, color='red', linewidth=0.5)

                #Formatta le date sull'asse x
                plt.gca().xaxis.set_major_locator(mdates.SecondLocator(interval=770))

                #Tick per i giorni con colore differenti
                plt.gca().xaxis.set_minor_formatter(mdates.DateFormatter('%m-%d'))
                plt.gca().xaxis.set_minor_locator(mdates.DayLocator(interval=1))
                plt.tick_params(axis='x', which='minor', colors='red')

                # Impostazioni dell'asse y con tick ogni 100 unità
                plt.gca().yaxis.set_major_locator(plt.MultipleLocator(100))
                max_value = df['consumption'].max()
                if max_value>1000:
                    plt.gca().yaxis.set_minor_locator(plt.MultipleLocator(50))
                else:
                    plt.gca().yaxis.set_minor_locator(plt.MultipleLocator(20))
                plt.tick_params(axis='y', which='minor', colors='red')


                #Impostazioni della griglia
                plt.xlabel('Date')
                plt.ylabel('Consumption (L)')
                plt.title(f'Emissioni del pattern {key}')
                plt.legend()
                plt.grid(True, which='major', linestyle='-', linewidth=1)
                plt.grid(True, which='minor', linestyle='-', linewidth=5)
                plt.grid(True, which='major', axis='y', linestyle='-', linewidth=1)
                plt.grid(True, which='minor', axis='y', linestyle='-', linewidth=1)
                plt.xticks(rotation=90)

                pdf.savefig() #X CONTROLLI GRAFIIC

                # Chiudere il grafico per liberare memoria
                plt.close()
        #print(f'Fine per {key}')


In [ ]:
#verifica sulle intensita delle emissioni
df_dispositivo_emissioni=read_fixture('Washbasin')
#CALCOLI DELLA MEDIA MENSILE, TRIMESTRALE, SEMESTRALE
mm_array=calcola_media_mensile(df_dispositivo_emissioni)
mt_array=calcola_media_trimestrale(df_dispositivo_emissioni)
ms_array=calcola_media_semestrale(df_dispositivo_emissioni)
ma=calcola_media_annuale(df_dispositivo_emissioni)
#DEBUG 
print(f'Media MENSILE: {mm_array}\n')
print(f'Media TRIMESTRALE: {mt_array}\n')
print(f'Media SEMESTRALE: {ms_array}\n')
print(f'Media ANNUALE: {ma}\n')
temp=(ma/100)*0.1
print(f'errore: {temp}\n')

date=[('2019-02-13') ,('2019-06-01'),('2019-08-14'),('2019-03-18'),('2019-04-24'),('2019-04-25'),('2019-02-22'),('2019-07-14')
        ,('2019-03-19'),('2019-03-20'),('2019-04-06'),('2019-05-24'),('2019-04-11'),('2019-05-29'),('2019-04-16'),('2019-04-26') 
        ,('2019-05-18'),('2019-06-14'),('2019-05-27'),('2019-09-01'),
        ('2019-09-23'),('2019-09-24'),('2019-10-22'),('2019-10-26'),('2019-04-14'),('2019-04-21'),('2019-04-30'),('2019-07-19'),('2019-09-08')]
for day in date:
    #print(f'{date}\n')
    start_date = pd.Timestamp(day)
    end_date = pd.Timestamp(day).replace(hour=23, minute=59, second=59)

    #CAPISCE ALGORTIMO
    df= df_dispositivo_emissioni[start_date:end_date]
    day_emiss=df['consumption'].to_numpy()
    day_emiss=day_emiss[0]
    print(f'{start_date}[{day_emiss}]\n')

In [ ]:
diz={'0': 125, 'CAMALA1P_S0': 2, 'CCMDLB3PMM_S2f': 1, 'CHMHLH4_S0': 1, 'CGMHLF3MPP_S0': 1, 'CHMHLG2LM_S0': 1, 'CHMHLH8_S2c': 1, 'CFMHLE5_S1a': 1, 'CHMHLH9_S2a': 2, 'CGMHLE5_S0': 1, 'CFMGLD3MPP_S2f': 1, 'CHMHLG5_S1a': 1, 'CHMHLH8_M1c': 1, 'CGMHLE3LMP_S0': 1, 'CHMHLH6_M1d': 1, 'CEMHLE4_S2a': 1, 'CDMGLD5_S1f': 1, 'CFMHLF5_S2a': 2, 'CFMHLF2MP_S0': 1, 'CDMGLD3PPL_S0': 1, 'CCMDLC3MMP_S2a': 1, 'CCMDLC1M_S0': 1, 'CDMFLC3PPL_P1c': 1, 'CFMHLF7_S2f': 1, 'CAMBLA2PP_P1a': 1, 'CEMGLD3MPP_S2f': 1, 'CDMFLC5_S1f': 1, 'CEMHLE8_S2f': 1, 'CDMGLD3PPM_S0': 1, 'CAMALA1M_S0': 3, 'CDMGLD4_S0': 2, 'CDMGLD3LPP_S0': 1, 'CDMGLD13_M1d': 1, 'CEMHLE3PLM_S0': 1, 'CHMHLH8_S1a': 1, 'CDMFLD2MP_S0': 1, 'CHMHLH8_S2a': 1, 'CCMDLC2MM_S0': 1, 'CFMHLE8_S0': 1, 'CCMDLC2MP_S0': 1, 'CCMDLC2PP_S0': 1, 'CCMDLC2LP_S0': 1, 'CHMHLH3MPP_S0': 1, 'CBMBLC2PL_P1a': 1, 'CCMCLD1M_S0': 1, 'CDMCLE3MMM_S0': 1, 'CEMELF7_S2d': 1, 'CDMCLD5_S1f': 2, 'CFMELH2MM_S0': 1, 'CBMBLB7_S1a': 1, 'CCMBLC3MMM_S2f': 1, 'CBMBLC2MP_S0': 2, 'CEMELG6_S0': 1, 'CFMELH5_S0': 1, 'CAMALA2PP_S0': 5, 'CEMDLF5_S2a': 1, 'CDMDLE7_S2c': 2, 'CEMELF5_M1f': 1, 'CHMHLH3PMP_S2f': 1, 'CBMALB5_M1c': 1, 'CDMDLE7_S2f': 1, 'CFMFLH6_S0': 1, 'CHMHLH9_M1b': 1, 'CHMGLH7_S2c': 1, 'CCMCLD7_S2c': 1, 'CCMCLD2MM_P1f': 1, 'CHMHLH6_S1a': 1, 'CCMBLC4_S0': 1, 'CDMCLD4_M1c': 1, 'CFMDLF6_S2f': 1, 'CCMBLC2PL_S0': 1, 'CDMCLE6_S2a': 1, 'CDMCLD3MPM_S0': 1, 'CEMDLE4_S2a': 1, 'CCMCLD4_S2a': 1, 'CCMCLD2MP_P1f': 1, 'CAMALA3PPP_S2a': 1, 'CDMCLE5_S1f': 1, 'CEMDLF4_S2f': 1, 'CDMCLD1L_S0': 1, 'CDMCLD4_S2f': 1, 'CHMHLH6_S2c': 2, 'CCMCLD9_S2c': 1, 'CBMBLB2PP_S0': 1, 'CGMFLH5_S1a': 1, 'CFMELG4_M1f': 1, 'CGMELG4_S2f': 1, 'CFMELG4_S2f': 1, 'CAMALA4_S2a': 2, 'CDMCLE1M_S0': 1, 'CHMFLH5_M1d': 1, 'CDMELG7_S1f': 1, 'CDMFLH7_M1b': 1, 'CDMFLH8_S1a': 1, 'CBMCLD4_S2a': 1, 'CCMDLF4_S2a': 1, 'CCMDLF9_S2c': 1, 'CBMCLD7_S1f': 1, 'CBMCLD8_S1a': 1, 'CFMHLH12_S2c': 1, 'CBMBLB4_S2f': 1, 'CEMFLH9_S2c': 1, 'CEMGLH8_M1c': 1, 'CHMHLH10_M1b': 1, 'CBMCLD7_S1a': 1, 'CCMDLF7_M1b': 1, 'CCMDLE4_S0': 1, 'CHMHLH11_M2d': 1, 'CBMBLC3MMM_P1c': 1, 'CEMFLH16_M1d': 1, 'CEMFLH11_S2c': 1, 'CBMCLD5_S1f': 1, 'CCMDLF2LM_S0': 1, 'CEMGLH11_S2d': 1, 'CCMCLE11_M2c': 1, 'CAMALA5_M1d': 1, 'CAMALA4_S0': 1, 'CDMELG8_S2a': 1, 'CEMELF1L_S0': 1, 'CAMBLB3MPP_S2a': 1, 'CEMFLG9_S1c': 1, 'CCMCLD2LM_S0': 1, 'CAMALA2MP_S0': 1, 'CHMHLH11_M1a': 1, 'CDMELF7_S1a': 1, 'CFMGLH4_S2a': 1, 'CDMDLE3PLP_S0': 1, 'CGMHLH13_S2d': 1, 'CCMDLD4_S2f': 1, 'CHMHLH11_S2a': 1, 'CEMELF2MP_P1f': 1, 'CDMDLE4_S2a': 1, 'CHMHLH10_S2c': 1, 'CCMCLD4_S2f': 1, 'CCMCLD8_S2c': 1, 'CCMDLD4_M1a': 1, 'CDMELF7_M1d': 1, 'CDMELF4_M1f': 1, 'CDMDLE5_S2c': 1, 'CAMALA5_S2a': 1, 'CCMDLD4_M1c': 1, 'CEMFLG6_S0': 1, 'CEMELF9_S1f': 1, 'CDMDLE7_M1b': 1, 'CFMGLH10_S1c': 1, 'CDMELF2PM_P1f': 1, 'CHMDLE6_M1d': 1, 'CHMHLH16_M1c': 1, 'CDMBLC5_S0': 1, 'CDMBLC1M_S0': 1, 'CCMBLB1P_S0': 1, 'CGMDLE4_S2f': 1, 'CDMBLC3PLP_S2a': 1, 'CFMDLD6_S2a': 1, 'CFMDLD10_M1e': 1, 'CCMBLB2PM_P1a': 1, 'CEMCLC4_P1c': 1, 'CHMFLH3LPP_P1c': 1, 'CDMBLB1M_S0': 1, 'CHMHLH2MP_S0': 1, 'CAMALA2MP_P1a': 1, 'CEMCLD4_S2a': 1, 'CHMHLH6_S2a': 1, 'CDMELF3MPM_S0': 1, 'CBMCLD5_S0': 1, 'CEMGLH10_S2b': 1, 'CDMELF5_S1a': 1, 'CBMCLC4_S2f': 1, 'CCMCLD3PPM_P1a': 1, 'CCMDLE2MM_S0': 1, 'CGMHLH4_S2f': 1, 'CBMCLC2PP_P1a': 1, 'CCMDLE5_S2c': 1, 'CGMHLH7_S2c': 1, 'CAMBLB3PPP_S0': 1, 'CCMCLD2MP_P1a': 1, 'CGMHLH10_M1d': 1, 'CEMGLH11_M1d': 1, 'CEMGLH6_M1a': 1, 'CDMELF2MM_S0': 1, 'CGMHLH8_S2a': 1, 'CCMDLE3PMP_S2f': 1, 'CHMHLH7_S1f': 2, 'CHMHLH8_S2f': 1, 'CFMHLH4_S2f': 1, 'CCMDLE6_S1a': 1, 'CHMHLH5_S0': 1, 'CGMHLH9_S1c': 1, 'CGMHLH8_M1a': 1, 'CEMFLH8_S2c': 1, 'CEMHLH4_S0': 1, 'CAMALA3MMP_P1c': 1, 'CBMELD2MM_S0': 1, 'CGMHLH6_M1c': 1, 'CDMHLH8_S1a': 1, 'CHMHLH4_S2a': 1, 'CBMFLE5_S0': 1, 'CHMHLH6_S1f': 1, 'CEMHLH3MPM_S2f': 1, 'CBMFLE3MPP_S2f': 1, 'CHMHLH10_S2d': 1, 'CHMHLH9_S1a': 1, 'CHMHLH5_S1a': 1, 'CCMHLG5_S1f': 1, 'CCMGLE3MPP_S0': 1, 'CCMHLG4_S0': 1, 'CCMHLG5_M1d': 1, 'CHMHLH8_M1d': 1, 'CAMCLC2PP_S0': 1, 'CBMDLC2PP_S0': 1, 'CEMHLH6_S0': 2, 'CBMFLE3LPP_S2f': 1, 'CCMHLG3LMP_S0': 1, 'CDMHLH4_S0': 1, 'CGMHLH6_S1a': 1, 'CFMHLH10_M1a': 1}

new_diz={}
for key,value in diz.items():
    if value in new_diz:#se lo trova 
        new_diz[value].append(key)
    else:
        new_diz[value]=[]
        new_diz[value].append(key)

for key,value in new_diz.items():
    print(f'{key}:\t{new_diz[key]}\n')


In [ ]:
#MAIN PER STAMPARE I PATTERN IN SETTIMANE

In [12]:
def scelta_pattern(df_final_sec,day_emiss,mm,mt,ms,num):
    #Intervalli sul grafico 770sec=12,8m, media valori ogni 30sec

    df_inter =df_final_sec.resample('770S').mean() #abbiamo solo emissioni 'importanti'
    
    #VEDE GLI INTERVALLI E DURATA
    in_interval = False
    start_date = None
    end_date = None
    duration=0
    intervals = []
    for date, consumption in df_inter['consumption'].items():
        if consumption > 0:#abbiamo delle emissioni
            if not in_interval:#inizio intervallo
                in_interval = True
                start_date = date
                duration = 1 
            else:#continuo intervallo
                duration += 1
        else:#no emissioni
            if in_interval:
                in_interval = False
                end_date=date
                intervals.append((start_date, end_date, duration))    
    if in_interval:
        end_date=date
        intervals.append((start_date, end_date, duration))
    #DEBUG print(intervals)

    #CREAZIONE PATTERN
    #ANALISI EMISSIONI 
    #print(f'{day_emiss }')
    #mm
    pattern=""
    if day_emiss<(mm/4):
        pattern+= "CA"
    elif day_emiss<(mm/2):
        pattern+= "CB"
    elif day_emiss<(mm*(3/4)):
        pattern+= "CC"
    elif day_emiss<mm:
        pattern+= "CD"
    elif day_emiss<(mm*(5/4)):
        pattern+= "CE"
    elif day_emiss<(mm*(3/2)):
        pattern+= "CF"
    elif day_emiss<(mm*(7/4)):
        pattern+= "CG"
    else:
        pattern+= "CH"
        
    #mt
    if day_emiss<(mt/4):
        pattern+= "MA"
    elif day_emiss<(mt/2):
        pattern+= "MB"
    elif day_emiss<(mt*(3/4)):
        pattern+= "MC"
    elif day_emiss<mt:
        pattern+= "MD"
    elif day_emiss<(mt*(5/4)):
        pattern+= "ME"
    elif day_emiss<(mt*(3/2)):
        pattern+= "MF"
    elif day_emiss<(mt*(7/4)):
        pattern+= "MG"
    else:
        pattern+= "MH"

    #ms
    if day_emiss<(ms / 4):
        pattern+= "LA"
    elif day_emiss<(ms / 2):
        pattern+= "LB"
    elif day_emiss<(ms * (3 / 4)):
        pattern+= "LC"
    elif day_emiss<ms:
        pattern+= "LD"
    elif day_emiss<(ms * (5 / 4)):
        pattern+= "LE"
    elif day_emiss<(ms * (3 / 2)):
        pattern+= "LF"
    elif day_emiss<(ms * (7 / 4)):
        pattern+= "LG"
    else:
        pattern+= "LH"
    
    #ANALISI INTERVALLI
    count_intervalli=len(intervals)
    pattern+= str(count_intervalli) #NUMERO DI INTERVALLI 

    #AGGIUNTA SULL'ANANLISI PER MENO DI 4 INTERVALLI
    if count_intervalli<4:#osservazione potrei farlo per tutti o allargare il numero 
        for _,_, durata in intervals:
            if durata>3: pattern += "L"#INTERVALLO LUNGO
            elif durata>1:  pattern += "M"#INTERVALLO MEDIO
            else: pattern += "P"#INTERVALLO PICCOLO     

    pattern += "_"   
    
    #CHANGE
    count_l=0
    count_m=0
    count_p=0
    for i in range(len(intervals) - 1):
        ris=int(( (intervals[i+1][0]).to_pydatetime() - (intervals[i][1]).to_pydatetime() ).total_seconds()/770)
        if ris>3: count_l += 1#intervallo di tempo LUNGO
        elif ris>1:  count_m += 1#intervallo di tempo MEDIO
        else: count_p += 1#intervallo di tempo PICCOLO
        #diff_tra_intervalli.append(  ris )
    
    #debug print count
    #if num==5: print(f'lungo={count_l} medio={count_m} piccolo={count_p}\n')

    tot=count_l+(count_m+count_p)
    #ANALISI GENERALE INTERVALLI
    if count_l==tot: pattern+="S0"#MOLTO MOLTO SPARSO
    elif count_l>=(3/4)*tot:#MOLTO SPARSO
        if count_m==(count_m+count_p): pattern+="S1a"
        elif count_m>=(3/4)*(count_m+count_p): pattern+="S1b"
        elif count_m>=(1/2)*(count_m+count_p): pattern+="S1c"
        elif count_m>=(1/4)*(count_m+count_p): pattern+="S1d"
        elif count_m>0: pattern+="S1e"
        elif count_m==0: pattern+="S1f"
    elif count_l>=(1/2)*tot:#SPARSO
        if count_m==(count_m+count_p): pattern+="S2a"
        elif count_m>=(3/4)*(count_m+count_p): pattern+="S2b"
        elif count_m>=(1/2)*(count_m+count_p): pattern+="S2c"
        elif count_m>=(1/4)*(count_m+count_p): pattern+="S2d"
        elif count_m>0: pattern+="S2e"
        elif count_m==0: pattern+="S2f"
    elif count_l>=(1/4)*tot:#MEDIAMENTE SPARSO
        if count_m==(count_m+count_p): pattern+="M1a"
        elif count_m>=(3/4)*(count_m+count_p): pattern+="M1b"
        elif count_m>=(1/2)*(count_m+count_p): pattern+="M1c"
        elif count_m>=(1/4)*(count_m+count_p): pattern+="M1d"
        elif count_m>0: pattern+="M1e"
        elif count_m==0: pattern+="M1f"
    elif count_l>0:#NON MOLTO SPARSO
        if count_m==(count_m+count_p): pattern+="M2a"
        elif count_m>=(3/4)*(count_m+count_p): pattern+="M2b"
        elif count_m>=(1/2)*(count_m+count_p): pattern+="M2c"
        elif count_m>=(1/4)*(count_m+count_p): pattern+="M2d"
        elif count_m>0: pattern+="M2e"
        elif count_m==0: pattern+="P0" #CONCENTRATO
    elif count_l==0:#NON MOLTO SPARSO
        if count_m==(count_m+count_p): pattern+="P1a"
        elif count_m>=(3/4)*(count_m+count_p): pattern+="P1b"
        elif count_m>=(1/2)*(count_m+count_p): pattern+="P1c"
        elif count_m>=(1/4)*(count_m+count_p): pattern+="P1d"
        elif count_m>0: pattern+="P1e"
        elif count_m==0: pattern+="P1f" #CONCENTRATO
    #print("DEBUG")
    return pattern

In [13]:
def scelta_pattern_medie_tre(diz_patterns ,mm_array,mt_array,ms_array,ma,array_emissioni,df_final_emissioni,df_final_sec,mese_start,giorno_cambio_mese,weekstart_date,num):
    #array_emissioni=array dei consumi
    #CARTELLA DOVE SALVARE I GRAFICI
    array_pattern = []
    perc_val=5 #CAMBIARE IL VALORE PER AUMENTARE/DIMINUIRE IL MINIMO ACCETTABILE PER OGNI INTERVALLO
    # codice che sceglie pattern  

    media_m=mm_array[mese_start-1]
    #trimestrale
    if (mese_start)<=3: media_t=mt_array[0]
    elif (mese_start)<=6: media_t=mt_array[1]
    elif (mese_start)<=9: media_t=mt_array[2]
    else: media_t=mt_array[3]
    #semestrale
    if (mese_start)<=6: media_s=ms_array[0]
    else: media_s=ms_array[1]

    
    for i in range(len(array_emissioni) ):
        
        #CONTROLLA SE STIAMO IN UN NUOVO MESE
        if giorno_cambio_mese==i:
            #mensile
            media_m=mm_array[mese_start]
            #trimestrale
            if (mese_start+1)<=3: media_t=mm_array[0]
            elif (mese_start+1)<=6: media_t=mm_array[1]
            elif (mese_start+1)<=9: media_t=mm_array[2]
            else: media_t=mm_array[3]
            #semestrale
            if (mese_start+1)<=6: media_s=ms_array[0]
            else: media_s=ms_array[1]
            
        pattern_day=""

        #***********CONTROLLA SE SUPERIAMO LO 0.1% DELLA MEDIA ANNUALE DI EMISSIONI NELLA GIORNATA 
        day_emiss=array_emissioni[i]
        temp=(ma/100)*0.1
        if day_emiss>=temp:
            #**********
            start_date=weekstart_date+pd.DateOffset(days=i)
            end_date=pd.Timestamp(start_date).replace(hour=23, minute=59, second=59)
            df=df_final_sec[start_date:end_date]

            df_max =df.resample('30S').mean()
            max_value = df_max['consumption'].max()
            und_value=(max_value/100)*perc_val
            #ELIMINO TUTTE LE EMISSIONI INFERIORI A UNA CERTA SOGLIA
            for index, row in df.iterrows():
                if row['consumption'] < und_value:
                    df.at[index, 'consumption'] = 0

            #ALGORITMO
            pattern_day=scelta_pattern(df,day_emiss,media_m,media_t,media_s,num)
            array_pattern.append(pattern_day)
            if pattern_day not in diz_patterns:#se non lo trova 
                diz_patterns[pattern_day] = len(diz_patterns)+1
            #debug controllo grafici per corrispettivi pattern
            #if num==5:
            #    graf=df.resample('30S').mean()
            #    plt.figure(figsize=(20, 6))

                #Grafico per tutte le emissioni
            #    plt.plot(graf.index, graf['consumption'], label={'consumption': 'VolumeMedia [L]'}, color='red', linewidth=0.5)

                #Formatta le date sull'asse x
            #    plt.gca().xaxis.set_major_locator(mdates.SecondLocator(interval=770))

                #Tick per i giorni con colore differenti
            #    plt.gca().xaxis.set_minor_formatter(mdates.DateFormatter('%m-%d'))
            #    plt.gca().xaxis.set_minor_locator(mdates.DayLocator(interval=1))
            #    plt.tick_params(axis='x', which='minor', colors='red')

                # Impostazioni dell'asse y con tick ogni 100 unità
            #    plt.gca().yaxis.set_major_locator(plt.MultipleLocator(100))
            #    if max_value>1000:
            #        plt.gca().yaxis.set_minor_locator(plt.MultipleLocator(50))
            #    else:
            #        plt.gca().yaxis.set_minor_locator(plt.MultipleLocator(20))
             #   plt.tick_params(axis='y', which='minor', colors='red')


                #Impostazioni della griglia
            #    plt.xlabel('Date')
            #    plt.ylabel('Consumption (L)')
            #    plt.title(f'Emissioni del pattern {start_date}')
            #    plt.legend()
            #    plt.grid(True, which='major', linestyle='-', linewidth=1)
            #    plt.grid(True, which='minor', linestyle='-', linewidth=5)
            #    plt.grid(True, which='major', axis='y', linestyle='-', linewidth=1)
            #    plt.grid(True, which='minor', axis='y', linestyle='-', linewidth=1)
            #    plt.xticks(rotation=90)

            #    plt.show() #X CONTROLLI GRAFIIC

                # Chiudere il grafico per liberare memoria
            #    plt.close()
        else:
            array_pattern.append("0")
            if "0" not in diz_patterns:#se non lo trova 
                diz_patterns["0"] = len(diz_patterns)+1
            
        
    # Stampa l'array pattern e emissioni
    print(f"Array con i pattern della settimana {weekstart_date} da LUN a DOM [{num}]")
    print(f"{array_pattern}")
    print(f"{array_emissioni}\n")

In [14]:
def ciclo_pattern_settimane_precise_medie_tre(df_dispositivo_emissioni,df_dispositivo_sec,mm_array,mt_array,ms_array,ma):
    num=0 #numero settimana
    start_date = pd.Timestamp('2019-01-01')
    end_date = pd.Timestamp('2019-12-31')
    diz_patterns={}
    #trovo il prossimo lunedi
    start_date += pd.DateOffset(days=(7 - start_date.weekday())) #0=lunedi 6=domenica
    current_date=start_date
    while current_date <= end_date:
        weekstart_date=current_date
        weekend_date=weekstart_date + pd.DateOffset(days=6)
        weekend_date=pd.Timestamp(weekend_date).replace(hour=23, minute=59, second=59)
        if weekend_date > end_date:
            break
        # Creare un range di date
        date_rng_sec = pd.date_range(start= weekstart_date, end=weekend_date, freq='S')
        date_rng = pd.date_range(start= weekstart_date, end=weekend_date, freq='D')
        current_date += pd.DateOffset(weeks=1)

        # Creare un DataFrame vuoto con il range di date specificato
        df_final_emissioni = pd.DataFrame(date_rng, columns=['date'])
        df_final_emissioni.set_index('date', inplace=True)

        df_final_sec = pd.DataFrame(date_rng_sec, columns=['date'])
        df_final_sec.set_index('date', inplace=True)
        
        # Unire il DataFrame giornaliero con il DataFrame vuoto per assicurarsi che tutte le date siano presenti
        df_final_emissioni = df_final_emissioni.join(df_dispositivo_emissioni, how='left').fillna(0)
        df_final_sec = df_final_sec.join(df_dispositivo_sec, how='left').fillna(0)
        
        array_emissioni = df_final_emissioni['consumption'].to_numpy()

        # Controlliamo se ci troviamo in una settimana tra due mesi, causa controlli con le medie
        mese_start = weekstart_date.month
        giorno_cambio_mese = -1
        for i in range(7):
            current_day = weekstart_date + pd.DateOffset(days=i)
            if current_day.month != mese_start:
                giorno_cambio_mese = i #gia sappiamo che se cambia mese aumenta mese_start di 1
                break
        
        #campionamento per secondi
        scelta_pattern_medie_tre(diz_patterns,mm_array,mt_array,ms_array,ma,array_emissioni,df_final_emissioni,df_final_sec,mese_start,giorno_cambio_mese,weekstart_date,num)
        num=num+1
        #per velocizzare i test mi fermo alla settimana numero 7
        #if num==7:
        #    break
    print(diz_patterns)


In [15]:
df_dispositivo_sec=read_fixture_sec('Washbasin')
df_dispositivo_emissioni=read_fixture('Washbasin')


#CALCOLI DELLA MEDIA MENSILE, TRIMESTRALE, SEMESTRALE
mm_array=calcola_media_mensile(df_dispositivo_emissioni)
mt_array=calcola_media_trimestrale(df_dispositivo_emissioni)
ms_array=calcola_media_semestrale(df_dispositivo_emissioni)
ma=df_dispositivo_emissioni['consumption'].mean()
#DEBUG 
#print(f'Media MENSILE: {mm_array}\n')
#print(f'Media TRIMESTRALE: {mt_array}\n')
#print(f'Media SEMESTRALE: {ms_array}\n')
#print(f'Media ANNUALE: {ma}\n')

ciclo_pattern_settimane_precise_medie_tre(df_dispositivo_emissioni,df_dispositivo_sec,mm_array,mt_array,ms_array,ma)

Array con i pattern della settimana 2019-01-07 00:00:00 da LUN a DOM [0]
['0', '0', '0', '0', '0', '0', '0']
[0. 0. 0. 0. 0. 0. 0.]

Array con i pattern della settimana 2019-01-14 00:00:00 da LUN a DOM [1]
['0', '0', '0', '0', '0', '0', '0']
[0. 0. 0. 0. 0. 0. 0.]

Array con i pattern della settimana 2019-01-21 00:00:00 da LUN a DOM [2]
['0', '0', '0', '0', '0', '0', '0']
[0. 0. 0. 0. 0. 0. 0.]

Array con i pattern della settimana 2019-01-28 00:00:00 da LUN a DOM [3]
['0', '0', '0', '0', '0', '0', '0']
[0. 0. 0. 0. 0. 0. 0.]

Array con i pattern della settimana 2019-02-04 00:00:00 da LUN a DOM [4]
['0', '0', '0', '0', '0', '0', '0']
[0. 0. 0. 0. 0. 0. 0.]

Array con i pattern della settimana 2019-02-11 00:00:00 da LUN a DOM [5]
['0', '0', 'CAMALA1P_S0', '0', '0', 'CCMDLB3PMM_S2f', 'CHMHLH4_S0']
[    0.     0.   177.     0.     0.  3064. 12306.]

Array con i pattern della settimana 2019-02-18 00:00:00 da LUN a DOM [6]
['CGMHLF3MPP_S0', 'CHMHLG2LM_S0', 'CHMHLH8_S2c', 'CFMHLE5_S1a', 'CHMH

In [4]:
men=['CA','CB','CC' ,'CD','CE' ,'CF' ,'CG','CH']

tri=['MA','MB','MC' ,'MD' ,'ME', 'MF' ,'MG' ,'MH']

sem=['LA','LB','LC' ,'LD','LE' ,'LF' ,'LG','LH']

pattern=[]
for elem in men:
    for elemtri in tri:
        for elemsem in sem:
            pattern.append(elem+elemtri+elemsem)
print(pattern)



['CAMALA', 'CAMALB', 'CAMALC', 'CAMALD', 'CAMALE', 'CAMALF', 'CAMALG', 'CAMALH', 'CAMBLA', 'CAMBLB', 'CAMBLC', 'CAMBLD', 'CAMBLE', 'CAMBLF', 'CAMBLG', 'CAMBLH', 'CAMCLA', 'CAMCLB', 'CAMCLC', 'CAMCLD', 'CAMCLE', 'CAMCLF', 'CAMCLG', 'CAMCLH', 'CAMDLA', 'CAMDLB', 'CAMDLC', 'CAMDLD', 'CAMDLE', 'CAMDLF', 'CAMDLG', 'CAMDLH', 'CAMELA', 'CAMELB', 'CAMELC', 'CAMELD', 'CAMELE', 'CAMELF', 'CAMELG', 'CAMELH', 'CAMFLA', 'CAMFLB', 'CAMFLC', 'CAMFLD', 'CAMFLE', 'CAMFLF', 'CAMFLG', 'CAMFLH', 'CAMGLA', 'CAMGLB', 'CAMGLC', 'CAMGLD', 'CAMGLE', 'CAMGLF', 'CAMGLG', 'CAMGLH', 'CAMHLA', 'CAMHLB', 'CAMHLC', 'CAMHLD', 'CAMHLE', 'CAMHLF', 'CAMHLG', 'CAMHLH', 'CBMALA', 'CBMALB', 'CBMALC', 'CBMALD', 'CBMALE', 'CBMALF', 'CBMALG', 'CBMALH', 'CBMBLA', 'CBMBLB', 'CBMBLC', 'CBMBLD', 'CBMBLE', 'CBMBLF', 'CBMBLG', 'CBMBLH', 'CBMCLA', 'CBMCLB', 'CBMCLC', 'CBMCLD', 'CBMCLE', 'CBMCLF', 'CBMCLG', 'CBMCLH', 'CBMDLA', 'CBMDLB', 'CBMDLC', 'CBMDLD', 'CBMDLE', 'CBMDLF', 'CBMDLG', 'CBMDLH', 'CBMELA', 'CBMELB', 'CBMELC', 'CBMELD',

In [5]:
print(len(['CAMALA', 'CAMALB', 'CAMALC', 'CAMALD', 'CAMALE', 'CAMALF', 'CAMALG', 'CAMALH', 'CAMBLA', 'CAMBLB', 'CAMBLC', 'CAMBLD', 'CAMBLE', 'CAMBLF', 'CAMBLG', 'CAMBLH', 'CAMCLA', 'CAMCLB', 'CAMCLC', 'CAMCLD', 'CAMCLE', 'CAMCLF', 'CAMCLG', 'CAMCLH', 'CAMDLA', 'CAMDLB', 'CAMDLC', 'CAMDLD', 'CAMDLE', 'CAMDLF', 'CAMDLG', 'CAMDLH', 'CAMELA', 'CAMELB', 'CAMELC', 'CAMELD', 'CAMELE', 'CAMELF', 'CAMELG', 'CAMELH', 'CAMFLA', 'CAMFLB', 'CAMFLC', 'CAMFLD', 'CAMFLE', 'CAMFLF', 'CAMFLG', 'CAMFLH', 'CAMGLA', 'CAMGLB', 'CAMGLC', 'CAMGLD', 'CAMGLE', 'CAMGLF', 'CAMGLG', 'CAMGLH', 'CAMHLA', 'CAMHLB', 'CAMHLC', 'CAMHLD', 'CAMHLE', 'CAMHLF', 'CAMHLG', 'CAMHLH', 'CBMALA', 'CBMALB', 'CBMALC', 'CBMALD', 'CBMALE', 'CBMALF', 'CBMALG', 'CBMALH', 'CBMBLA', 'CBMBLB', 'CBMBLC', 'CBMBLD', 'CBMBLE', 'CBMBLF', 'CBMBLG', 'CBMBLH', 'CBMCLA', 'CBMCLB', 'CBMCLC', 'CBMCLD', 'CBMCLE', 'CBMCLF', 'CBMCLG', 'CBMCLH', 'CBMDLA', 'CBMDLB', 'CBMDLC', 'CBMDLD', 'CBMDLE', 'CBMDLF', 'CBMDLG', 'CBMDLH', 'CBMELA', 'CBMELB', 'CBMELC', 'CBMELD', 'CBMELE', 'CBMELF', 'CBMELG', 'CBMELH', 'CBMFLA', 'CBMFLB', 'CBMFLC', 'CBMFLD', 'CBMFLE', 'CBMFLF', 'CBMFLG', 'CBMFLH', 'CBMGLA', 'CBMGLB', 'CBMGLC', 'CBMGLD', 'CBMGLE', 'CBMGLF', 'CBMGLG', 'CBMGLH', 'CBMHLA', 'CBMHLB', 'CBMHLC', 'CBMHLD', 'CBMHLE', 'CBMHLF', 'CBMHLG', 'CBMHLH', 'CCMALA', 'CCMALB', 'CCMALC', 'CCMALD', 'CCMALE', 'CCMALF', 'CCMALG', 'CCMALH', 'CCMBLA', 'CCMBLB', 'CCMBLC', 'CCMBLD', 'CCMBLE', 'CCMBLF', 'CCMBLG', 'CCMBLH', 'CCMCLA', 'CCMCLB', 'CCMCLC', 'CCMCLD', 'CCMCLE', 'CCMCLF', 'CCMCLG', 'CCMCLH', 'CCMDLA', 'CCMDLB', 'CCMDLC', 'CCMDLD', 'CCMDLE', 'CCMDLF', 'CCMDLG', 'CCMDLH', 'CCMELA', 'CCMELB', 'CCMELC', 'CCMELD', 'CCMELE', 'CCMELF', 'CCMELG', 'CCMELH', 'CCMFLA', 'CCMFLB', 'CCMFLC', 'CCMFLD', 'CCMFLE', 'CCMFLF', 'CCMFLG', 'CCMFLH', 'CCMGLA', 'CCMGLB', 'CCMGLC', 'CCMGLD', 'CCMGLE', 'CCMGLF', 'CCMGLG', 'CCMGLH', 'CCMHLA', 'CCMHLB', 'CCMHLC', 'CCMHLD', 'CCMHLE', 'CCMHLF', 'CCMHLG', 'CCMHLH', 'CDMALA', 'CDMALB', 'CDMALC', 'CDMALD', 'CDMALE', 'CDMALF', 'CDMALG', 'CDMALH', 'CDMBLA', 'CDMBLB', 'CDMBLC', 'CDMBLD', 'CDMBLE', 'CDMBLF', 'CDMBLG', 'CDMBLH', 'CDMCLA', 'CDMCLB', 'CDMCLC', 'CDMCLD', 'CDMCLE', 'CDMCLF', 'CDMCLG', 'CDMCLH', 'CDMDLA', 'CDMDLB', 'CDMDLC', 'CDMDLD', 'CDMDLE', 'CDMDLF', 'CDMDLG', 'CDMDLH', 'CDMELA', 'CDMELB', 'CDMELC', 'CDMELD', 'CDMELE', 'CDMELF', 'CDMELG', 'CDMELH', 'CDMFLA', 'CDMFLB', 'CDMFLC', 'CDMFLD', 'CDMFLE', 'CDMFLF', 'CDMFLG', 'CDMFLH', 'CDMGLA', 'CDMGLB', 'CDMGLC', 'CDMGLD', 'CDMGLE', 'CDMGLF', 'CDMGLG', 'CDMGLH', 'CDMHLA', 'CDMHLB', 'CDMHLC', 'CDMHLD', 'CDMHLE', 'CDMHLF', 'CDMHLG', 'CDMHLH', 'CEMALA', 'CEMALB', 'CEMALC', 'CEMALD', 'CEMALE', 'CEMALF', 'CEMALG', 'CEMALH', 'CEMBLA', 'CEMBLB', 'CEMBLC', 'CEMBLD', 'CEMBLE', 'CEMBLF', 'CEMBLG', 'CEMBLH', 'CEMCLA', 'CEMCLB', 'CEMCLC', 'CEMCLD', 'CEMCLE', 'CEMCLF', 'CEMCLG', 'CEMCLH', 'CEMDLA', 'CEMDLB', 'CEMDLC', 'CEMDLD', 'CEMDLE', 'CEMDLF', 'CEMDLG', 'CEMDLH', 'CEMELA', 'CEMELB', 'CEMELC', 'CEMELD', 'CEMELE', 'CEMELF', 'CEMELG', 'CEMELH', 'CEMFLA', 'CEMFLB', 'CEMFLC', 'CEMFLD', 'CEMFLE', 'CEMFLF', 'CEMFLG', 'CEMFLH', 'CEMGLA', 'CEMGLB', 'CEMGLC', 'CEMGLD', 'CEMGLE', 'CEMGLF', 'CEMGLG', 'CEMGLH', 'CEMHLA', 'CEMHLB', 'CEMHLC', 'CEMHLD', 'CEMHLE', 'CEMHLF', 'CEMHLG', 'CEMHLH', 'CFMALA', 'CFMALB', 'CFMALC', 'CFMALD', 'CFMALE', 'CFMALF', 'CFMALG', 'CFMALH', 'CFMBLA', 'CFMBLB', 'CFMBLC', 'CFMBLD', 'CFMBLE', 'CFMBLF', 'CFMBLG', 'CFMBLH', 'CFMCLA', 'CFMCLB', 'CFMCLC', 'CFMCLD', 'CFMCLE', 'CFMCLF', 'CFMCLG', 'CFMCLH', 'CFMDLA', 'CFMDLB', 'CFMDLC', 'CFMDLD', 'CFMDLE', 'CFMDLF', 'CFMDLG', 'CFMDLH', 'CFMELA', 'CFMELB', 'CFMELC', 'CFMELD', 'CFMELE', 'CFMELF', 'CFMELG', 'CFMELH', 'CFMFLA', 'CFMFLB', 'CFMFLC', 'CFMFLD', 'CFMFLE', 'CFMFLF', 'CFMFLG', 'CFMFLH', 'CFMGLA', 'CFMGLB', 'CFMGLC', 'CFMGLD', 'CFMGLE', 'CFMGLF', 'CFMGLG', 'CFMGLH', 'CFMHLA', 'CFMHLB', 'CFMHLC', 'CFMHLD', 'CFMHLE', 'CFMHLF', 'CFMHLG', 'CFMHLH', 'CGMALA', 'CGMALB', 'CGMALC', 'CGMALD', 'CGMALE', 'CGMALF', 'CGMALG', 'CGMALH', 'CGMBLA', 'CGMBLB', 'CGMBLC', 'CGMBLD', 'CGMBLE', 'CGMBLF', 'CGMBLG', 'CGMBLH', 'CGMCLA', 'CGMCLB', 'CGMCLC', 'CGMCLD', 'CGMCLE', 'CGMCLF', 'CGMCLG', 'CGMCLH', 'CGMDLA', 'CGMDLB', 'CGMDLC', 'CGMDLD', 'CGMDLE', 'CGMDLF', 'CGMDLG', 'CGMDLH', 'CGMELA', 'CGMELB', 'CGMELC', 'CGMELD', 'CGMELE', 'CGMELF', 'CGMELG', 'CGMELH', 'CGMFLA', 'CGMFLB', 'CGMFLC', 'CGMFLD', 'CGMFLE', 'CGMFLF', 'CGMFLG', 'CGMFLH', 'CGMGLA', 'CGMGLB', 'CGMGLC', 'CGMGLD', 'CGMGLE', 'CGMGLF', 'CGMGLG', 'CGMGLH', 'CGMHLA', 'CGMHLB', 'CGMHLC', 'CGMHLD', 'CGMHLE', 'CGMHLF', 'CGMHLG', 'CGMHLH', 'CHMALA', 'CHMALB', 'CHMALC', 'CHMALD', 'CHMALE', 'CHMALF', 'CHMALG', 'CHMALH', 'CHMBLA', 'CHMBLB', 'CHMBLC', 'CHMBLD', 'CHMBLE', 'CHMBLF', 'CHMBLG', 'CHMBLH', 'CHMCLA', 'CHMCLB', 'CHMCLC', 'CHMCLD', 'CHMCLE', 'CHMCLF', 'CHMCLG', 'CHMCLH', 'CHMDLA', 'CHMDLB', 'CHMDLC', 'CHMDLD', 'CHMDLE', 'CHMDLF', 'CHMDLG', 'CHMDLH', 'CHMELA', 'CHMELB', 'CHMELC', 'CHMELD', 'CHMELE', 'CHMELF', 'CHMELG', 'CHMELH', 'CHMFLA', 'CHMFLB', 'CHMFLC', 'CHMFLD', 'CHMFLE', 'CHMFLF', 'CHMFLG', 'CHMFLH', 'CHMGLA', 'CHMGLB', 'CHMGLC', 'CHMGLD', 'CHMGLE', 'CHMGLF', 'CHMGLG', 'CHMGLH', 'CHMHLA', 'CHMHLB', 'CHMHLC', 'CHMHLD', 'CHMHLE', 'CHMHLF', 'CHMHLG', 'CHMHLH']))

512


In [20]:
array_annuale=[
    ['0', '0', '0', '0', '0', '0', '0'],
    ['0', '0', '0', '0', '0', '0', '0'],
    ['0', '0', '0', '0', '0', '0', '0'],
    ['0', '0', '0', '0', '0', '0', '0'],
    ['0', '0', '0', '0', '0', '0', '0'],
    ['0', '0', 'CAMALA1P_S0', '0', '0', 'CCMDLB3PMM_S2f', 'CHMHLH4_S0'],
    ['CGMHLF3MPP_S0', 'CHMHLG2LM_S0', 'CHMHLH8_S2c', 'CFMHLE5_S1a', 'CHMHLH9_S2a', 'CGMHLE5_S0', 'CFMGLD3MPP_S2f'],
    ['CHMHLG5_S1a', 'CHMHLH8_M1c', 'CGMHLE3LMP_S0', 'CHMHLH6_M1d', 'CEMHLE4_S2a', 'CDMHLD5_S1f', 'CFMHLF5_S2a'],
    ['CFMHLF2MP_S0', 'CDMGLD3PPL_S0', 'CCMDLC3MMP_S2a', 'CCMDLC1M_S0', '0', '0', 'CDMFLC3PPL_P1c'],
    ['CFMHLF7_S2f', 'CAMBLA2PP_P1a', 'CEMGLD3MPP_S2f', 'CDMFLC5_S1f', 'CEMHLE8_S2f', 'CFMHLF5_S2a', 'CDMGLD3PPM_S0'],
    ['CAMALA1M_S0', 'CDMGLD4_S0', 'CDMGLD4_S0', 'CDMGLD3LPP_S0', 'CDMGLD13_M1d', 'CEMHLE3PLM_S0', 'CHMHLH8_S1a'],
    ['CDMFLD2MP_S0', 'CHMHLH8_S2a', 'CCMDLC2MM_S0', 'CFMHLE8_S0', 'CCMDLC2MP_S0', 'CCMDLC2PP_S0', 'CCMDLC2LP_S0'],
    ['CHMHLH3MPP_S0', 'CBMBLC2PL_P1a', 'CCMCLD1M_S0', 'CDMCLE3MMM_S0', 'CEMELF7_S2d', 'CDMCLD5_S1f', 'CFMELH2MM_S0'],
    ['0', 'CBMBLB7_S1a', 'CCMBLC3MMM_S2f', 'CBMBLC2MP_S0', 'CEMELG6_S0', 'CFMELH5_S0', 'CAMALA2PP_S0'],
    ['CEMDLF5_S2a', 'CDMDLE7_S2c', 'CEMELF5_M1f', 'CHMHLH3PMP_S2f', 'CBMALB5_M1c', 'CDMDLE7_S2f', 'CAMALA2PP_S0'],
    ['CFMFLH6_S0', 'CHMHLH9_M1b', 'CAMALA1M_S0', 'CAMALA1M_S0', 'CDMDLE7_S2c', 'CHMGLH7_S2c', 'CCMCLD7_S2c'],
    ['CCMCLD2MM_P1f', 'CAMALA2PP_S0', 'CHMHLH6_S1a', 'CCMDLC4_S0', 'CDMELD4_M1c', 'CFMHLF6_S2f', 'CCMDLC2PL_S0'],
    ['CDMCLE6_S2a', 'CDMCLD3MPM_S0', 'CEMDLE4_S2a', 'CCMCLD4_S2a', 'CCMCLD2MP_P1f', '0', '0'],
    ['CAMALA3PPP_S2a', 'CDMCLE5_S1f', 'CEMDLF4_S2f', 'CDMCLD1L_S0', 'CDMCLD4_S2f', 'CHMHLH6_S2c', 'CCMCLD9_S2c'],
    ['CBMBLB2PP_S0', 'CGMFLH5_S1a', 'CFMELG4_M1f', 'CGMELG4_S2f', 'CDMCLD5_S1f', 'CFMELG4_S2f', '0'],
    ['CAMALA4_S2a', 'CDMCLE1M_S0', 'CBMBLC2MP_S0', 'CHMFLH5_M1d', '0', 'CAMALA1P_S0', 'CDMHLG7_S1f'],
    ['CDMFLH7_M1b', 'CDMFLH8_S1a', 'CBMCLD4_S2a', 'CCMDLF4_S2a', 'CCMDLF9_S2c', 'CBMCLD7_S1f', 'CBMCLD8_S1a'],
    ['CFMHLH12_S2c', 'CBMBLB4_S2f', 'CEMFLH9_S2c', 'CEMGLH8_M1c', 'CHMHLH6_S2c', 'CHMHLH10_M1b', 'CBMCLD7_S1a'],
    ['CCMDLF7_M1b', 'CCMDLE4_S0', 'CHMHLH11_M2d', 'CBMBLC3MMM_P1c', 'CEMFLH16_M1d', 'CEMFLH11_S2c', 'CBMCLD5_S1f'],
    ['CCMDLF2LM_S0', 'CEMGLH11_S2d', 'CCMCLE11_M2c', 'CAMALA5_M1d', '0', 'CAMALA4_S0', 'CDMELG8_S2a'],
    ['CEMELF1L_S0', 'CAMBLB3MPP_S2a', 'CEMFLG9_S1c', 'CCMCLD2LM_S0', 'CAMALA2MP_S0', 'CHMHLH11_M1a', 'CDMELF7_S1a'],
    ['CFMGLH4_S2a', 'CDMDLE3PLP_S0', 'CGMHLH13_S2d', 'CCMDLD4_S2f', 'CHMHLH11_S2a', 'CEMELF2MP_P1f', 'CHMHLH9_S2a'],
    ['CDMDLE4_S2a', 'CHMHLH10_S2c', 'CCMCLD4_S2f', 'CCMCLD8_S2c', 'CAMALA2PP_S0', '0', 'CCMDLD4_M1a'],
    ['CDMELF7_M1d', 'CDMELF4_M1f', 'CDMDLE5_S2c', 'CAMALA5_S2a', 'CCMDLD4_M1c', 'CEMFLG6_S0', 'CEMELF9_S1f'],
    ['CDMDLE7_M1b', 'CFMGLH10_S1c', 'CDMELF2PM_P1f', '0', '0', '0', 'CHMDLE6_M1d'],
    ['CHMHLH16_M1c', 'CDMBLC5_S0', 'CDMBLC1M_S0', 'CCMBLB1P_S0', 'CGMDLE4_S2f', '0', '0'],
    ['0', 'CDMBLC3PLP_S2a', '0', 'CFMDLD6_S2a', 'CFMDLD10_M1e', 'CCMBLB2PM_P1a', 'CEMCLC4_P1c'],
    ['CHMFLH3LPP_P1c', '0', '0', 'CDMBLB1M_S0', '0', '0', 'CHMHLH2MP_S0'],
    ['0', '0', '0', 'CAMALA2MP_P1a', 'CEMCLD4_S2a', 'CHMHLH6_S2a', 'CAMALA4_S2a'],
    ['CDMELF3MPM_S0', 'CBMCLD5_S0', 'CEMGLH10_S2b', 'CDMELF5_S1a', 'CBMCLC4_S2f', 'CCMCLD3PPM_P1a', 'CAMALA2PP_S0'],
    ['CCMDLE2MM_S0', 'CGMHLH4_S2f', 'CBMCLC2PP_P1a', '0', 'CCMDLE5_S2c', 'CGMHLH7_S2c', 'CAMBLB3PPP_S0'],
    ['CCMCLD2MP_P1a', 'CGMHLH10_M1d', 'CEMGLH11_M1d', 'CEMGLH6_M1a', 'CDMELF2MM_S0', 'CGMHLH8_S2a', 'CCMDLE3PMP_S2f'],
    ['CHMHLH7_S1f', 'CHMHLH8_S2f', 'CFMHLH4_S2f', 'CCMDLE6_S1a', 'CHMHLH5_S0', 'CGMHLH9_S1c', 'CGMHLH8_M1a'],
    ['CEMFLH8_S2c', 'CEMELH4_S0', 'CAMALA3MMP_P1c', 'CBMBLD2MM_S0', 'CGMHLH6_M1c', 'CDMDLH8_S1a', 'CHMHLH4_S2a'],
    ['CBMFLE5_S0', 'CHMHLH6_S1f', 'CEMHLH3MPM_S2f', 'CHMHLH7_S1f', 'CBMFLE3MPP_S2f', 'CHMHLH10_S2d', 'CHMHLH9_S1a'],
    ['CHMHLH5_S1a', 'CCMHLG5_S1f', 'CCMGLE3MPP_S0', 'CCMHLG4_S0', 'CCMHLG5_M1d', 'CHMHLH8_M1d', 'CAMCLC2PP_S0'],
    ['CBMDLC2PP_S0', 'CEMHLH6_S0', 'CBMFLE3LPP_S2f', 'CCMHLG3LMP_S0', 'CDMHLH4_S0', 'CEMHLH6_S0', 'CGMHLH6_S1a'],
    ['CFMHLH10_M1a', '0', '0', '0', '0', '0', '0'],
    ['0', '0', '0', '0', '0', '0', '0'],
    ['0', '0', '0', '0', '0', '0', '0'],
    ['0', '0', '0', '0', '0', '0', '0'],
    ['0', '0', '0', '0', '0', '0', '0'],
    ['0', '0', '0', '0', '0', '0', '0'],
    ['0', '0', '0', '0', '0', '0', '0'],
    ['0', '0', '0', '0', '0', '0', '0'],
    ['0', '0', '0', '0', '0', '0', '0']
]

diz_patterns={'0': 1, 'CAMALA1P_S0': 2, 'CCMDLB3PMM_S2f': 3, 'CHMHLH4_S0': 4, 'CGMHLF3MPP_S0': 5, 'CHMHLG2LM_S0': 6, 'CHMHLH8_S2c': 7, 'CFMHLE5_S1a': 8, 'CHMHLH9_S2a': 9, 'CGMHLE5_S0': 10, 'CFMGLD3MPP_S2f': 11, 'CHMHLG5_S1a': 12, 'CHMHLH8_M1c': 13, 'CGMHLE3LMP_S0': 14, 'CHMHLH6_M1d': 15, 'CEMHLE4_S2a': 16, 'CDMHLD5_S1f': 17, 'CFMHLF5_S2a': 18, 'CFMHLF2MP_S0': 19, 'CDMGLD3PPL_S0': 20, 'CCMDLC3MMP_S2a': 21, 'CCMDLC1M_S0': 22, 'CDMFLC3PPL_P1c': 23, 'CFMHLF7_S2f': 24, 'CAMBLA2PP_P1a': 25, 'CEMGLD3MPP_S2f': 26, 'CDMFLC5_S1f': 27, 'CEMHLE8_S2f': 28, 'CDMGLD3PPM_S0': 29, 'CAMALA1M_S0': 30, 'CDMGLD4_S0': 31, 'CDMGLD3LPP_S0': 32, 'CDMGLD13_M1d': 33, 'CEMHLE3PLM_S0': 34, 'CHMHLH8_S1a': 35, 'CDMFLD2MP_S0': 36, 'CHMHLH8_S2a': 37, 'CCMDLC2MM_S0': 38, 'CFMHLE8_S0': 39, 'CCMDLC2MP_S0': 40, 'CCMDLC2PP_S0': 41, 'CCMDLC2LP_S0': 42, 'CHMHLH3MPP_S0': 43, 'CBMBLC2PL_P1a': 44, 'CCMCLD1M_S0': 45, 'CDMCLE3MMM_S0': 46, 'CEMELF7_S2d': 47, 'CDMCLD5_S1f': 48, 'CFMELH2MM_S0': 49, 'CBMBLB7_S1a': 50, 'CCMBLC3MMM_S2f': 51, 'CBMBLC2MP_S0': 52, 'CEMELG6_S0': 53, 'CFMELH5_S0': 54, 'CAMALA2PP_S0': 55, 'CEMDLF5_S2a': 56, 'CDMDLE7_S2c': 57, 'CEMELF5_M1f': 58, 'CHMHLH3PMP_S2f': 59, 'CBMALB5_M1c': 60, 'CDMDLE7_S2f': 61, 'CFMFLH6_S0': 62, 'CHMHLH9_M1b': 63, 'CHMGLH7_S2c': 64, 'CCMCLD7_S2c': 65, 'CCMCLD2MM_P1f': 66, 'CHMHLH6_S1a': 67, 'CCMDLC4_S0': 68, 'CDMELD4_M1c': 69, 'CFMHLF6_S2f': 70, 'CCMDLC2PL_S0': 71, 'CDMCLE6_S2a': 72, 'CDMCLD3MPM_S0': 73, 'CEMDLE4_S2a': 74, 'CCMCLD4_S2a': 75, 'CCMCLD2MP_P1f': 76, 'CAMALA3PPP_S2a': 77, 'CDMCLE5_S1f': 78, 'CEMDLF4_S2f': 79, 'CDMCLD1L_S0': 80, 'CDMCLD4_S2f': 81, 'CHMHLH6_S2c': 82, 'CCMCLD9_S2c': 83, 'CBMBLB2PP_S0': 84, 'CGMFLH5_S1a': 85, 'CFMELG4_M1f': 86, 'CGMELG4_S2f': 87, 'CFMELG4_S2f': 88, 'CAMALA4_S2a': 89, 'CDMCLE1M_S0': 90, 'CHMFLH5_M1d': 91, 'CDMHLG7_S1f': 92, 'CDMFLH7_M1b': 93, 'CDMFLH8_S1a': 94, 'CBMCLD4_S2a': 95, 'CCMDLF4_S2a': 96, 'CCMDLF9_S2c': 97, 'CBMCLD7_S1f': 98, 'CBMCLD8_S1a': 99, 'CFMHLH12_S2c': 100, 'CBMBLB4_S2f': 101, 'CEMFLH9_S2c': 102, 'CEMGLH8_M1c': 103, 'CHMHLH10_M1b': 104, 'CBMCLD7_S1a': 105, 'CCMDLF7_M1b': 106, 'CCMDLE4_S0': 107, 'CHMHLH11_M2d': 108, 'CBMBLC3MMM_P1c': 109, 'CEMFLH16_M1d': 110, 'CEMFLH11_S2c': 111, 'CBMCLD5_S1f': 112, 'CCMDLF2LM_S0': 113, 'CEMGLH11_S2d': 114, 'CCMCLE11_M2c': 115, 'CAMALA5_M1d': 116, 'CAMALA4_S0': 117, 'CDMELG8_S2a': 118, 'CEMELF1L_S0': 119, 'CAMBLB3MPP_S2a': 120, 'CEMFLG9_S1c': 121, 'CCMCLD2LM_S0': 122, 'CAMALA2MP_S0': 123, 'CHMHLH11_M1a': 124, 'CDMELF7_S1a': 125, 'CFMGLH4_S2a': 126, 'CDMDLE3PLP_S0': 127, 'CGMHLH13_S2d': 128, 'CCMDLD4_S2f': 129, 'CHMHLH11_S2a': 130, 'CEMELF2MP_P1f': 131, 'CDMDLE4_S2a': 132, 'CHMHLH10_S2c': 133, 'CCMCLD4_S2f': 134, 'CCMCLD8_S2c': 135, 'CCMDLD4_M1a': 136, 'CDMELF7_M1d': 137, 'CDMELF4_M1f': 138, 'CDMDLE5_S2c': 139, 'CAMALA5_S2a': 140, 'CCMDLD4_M1c': 141, 'CEMFLG6_S0': 142, 'CEMELF9_S1f': 143, 'CDMDLE7_M1b': 144, 'CFMGLH10_S1c': 145, 'CDMELF2PM_P1f': 146, 'CHMDLE6_M1d': 147, 'CHMHLH16_M1c': 148, 'CDMBLC5_S0': 149, 'CDMBLC1M_S0': 150, 'CCMBLB1P_S0': 151, 'CGMDLE4_S2f': 152, 'CDMBLC3PLP_S2a': 153, 'CFMDLD6_S2a': 154, 'CFMDLD10_M1e': 155, 'CCMBLB2PM_P1a': 156, 'CEMCLC4_P1c': 157, 'CHMFLH3LPP_P1c': 158, 'CDMBLB1M_S0': 159, 'CHMHLH2MP_S0': 160, 'CAMALA2MP_P1a': 161, 'CEMCLD4_S2a': 162, 'CHMHLH6_S2a': 163, 'CDMELF3MPM_S0': 164, 'CBMCLD5_S0': 165, 'CEMGLH10_S2b': 166, 'CDMELF5_S1a': 167, 'CBMCLC4_S2f': 168, 'CCMCLD3PPM_P1a': 169, 'CCMDLE2MM_S0': 170, 'CGMHLH4_S2f': 171, 'CBMCLC2PP_P1a': 172, 'CCMDLE5_S2c': 173, 'CGMHLH7_S2c': 174, 'CAMBLB3PPP_S0': 175, 'CCMCLD2MP_P1a': 176, 'CGMHLH10_M1d': 177, 'CEMGLH11_M1d': 178, 'CEMGLH6_M1a': 179, 'CDMELF2MM_S0': 180, 'CGMHLH8_S2a': 181, 'CCMDLE3PMP_S2f': 182, 'CHMHLH7_S1f': 183, 'CHMHLH8_S2f': 184, 'CFMHLH4_S2f': 185, 'CCMDLE6_S1a': 186, 'CHMHLH5_S0': 187, 'CGMHLH9_S1c': 188, 'CGMHLH8_M1a': 189, 'CEMFLH8_S2c': 190, 'CEMELH4_S0': 191, 'CAMALA3MMP_P1c': 192, 'CBMBLD2MM_S0': 193, 'CGMHLH6_M1c': 194, 'CDMDLH8_S1a': 195, 'CHMHLH4_S2a': 196, 'CBMFLE5_S0': 197, 'CHMHLH6_S1f': 198, 'CEMHLH3MPM_S2f': 199, 'CBMFLE3MPP_S2f': 200, 'CHMHLH10_S2d': 201, 'CHMHLH9_S1a': 202, 'CHMHLH5_S1a': 203, 'CCMHLG5_S1f': 204, 'CCMGLE3MPP_S0': 205, 'CCMHLG4_S0': 206, 'CCMHLG5_M1d': 207, 'CHMHLH8_M1d': 208, 'CAMCLC2PP_S0': 209, 'CBMDLC2PP_S0': 210, 'CEMHLH6_S0': 211, 'CBMFLE3LPP_S2f': 212, 'CCMHLG3LMP_S0': 213, 'CDMHLH4_S0': 214, 'CGMHLH6_S1a': 215, 'CFMHLH10_M1a': 216}


array_new_pattern=[]
array_new_state=[]
singolo_new_pattern=[]
singolo_new_state=[]

for array_sett in array_annuale:
    patt_temp=[]
    state_temp=[]
    for i in range(len(array_sett) ):
        patt_temp.append(diz_patterns[array_sett[i]])
        if diz_patterns[array_sett[i]]==diz_patterns['0']: state_temp.append(1) 
        else: state_temp.append(2) #emissione maggiore di 0

        #array singolo per matlab
        singolo_new_pattern.append(diz_patterns[array_sett[i]])
        if diz_patterns[array_sett[i]]==diz_patterns['0']: singolo_new_state.append(1)
        else: singolo_new_state.append(2) #emissione maggiore di 0
     
    array_new_pattern.append(patt_temp)
    array_new_state.append(state_temp)

print("Array (patttern/stati):\n")
print(array_new_pattern)
print(array_new_state)
print("Array singoli (patttern/stati):\n")
print(singolo_new_pattern)
print(singolo_new_state)

Array (patttern/stati):

[[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1], [1, 1, 2, 1, 1, 3, 4], [5, 6, 7, 8, 9, 10, 11], [12, 13, 14, 15, 16, 17, 18], [19, 20, 21, 22, 1, 1, 23], [24, 25, 26, 27, 28, 18, 29], [30, 31, 31, 32, 33, 34, 35], [36, 37, 38, 39, 40, 41, 42], [43, 44, 45, 46, 47, 48, 49], [1, 50, 51, 52, 53, 54, 55], [56, 57, 58, 59, 60, 61, 55], [62, 63, 30, 30, 57, 64, 65], [66, 55, 67, 68, 69, 70, 71], [72, 73, 74, 75, 76, 1, 1], [77, 78, 79, 80, 81, 82, 83], [84, 85, 86, 87, 48, 88, 1], [89, 90, 52, 91, 1, 2, 92], [93, 94, 95, 96, 97, 98, 99], [100, 101, 102, 103, 82, 104, 105], [106, 107, 108, 109, 110, 111, 112], [113, 114, 115, 116, 1, 117, 118], [119, 120, 121, 122, 123, 124, 125], [126, 127, 128, 129, 130, 131, 9], [132, 133, 134, 135, 55, 1, 136], [137, 138, 139, 140, 141, 142, 143], [144, 145, 146, 1, 1, 1, 147], [148, 149, 150, 151, 152, 1, 1], [1, 153, 1, 154, 155, 156, 157], [158, 1, 1, 159, 1, 1

In [23]:
pattern_tot=['CAMALA', 'CAMALB', 'CAMALC', 'CAMALD', 'CAMALE', 'CAMALF', 'CAMALG', 'CAMALH', 'CAMBLA', 'CAMBLB', 'CAMBLC', 'CAMBLD', 'CAMBLE', 'CAMBLF', 'CAMBLG', 'CAMBLH', 'CAMCLA', 'CAMCLB', 'CAMCLC', 'CAMCLD', 'CAMCLE', 'CAMCLF', 'CAMCLG', 'CAMCLH', 'CAMDLA', 'CAMDLB', 'CAMDLC', 'CAMDLD', 'CAMDLE', 'CAMDLF', 'CAMDLG', 'CAMDLH', 'CAMELA', 'CAMELB', 'CAMELC', 'CAMELD', 'CAMELE', 'CAMELF', 'CAMELG', 'CAMELH', 'CAMFLA', 'CAMFLB', 'CAMFLC', 'CAMFLD', 'CAMFLE', 'CAMFLF', 'CAMFLG', 'CAMFLH', 'CAMGLA', 'CAMGLB', 'CAMGLC', 'CAMGLD', 'CAMGLE', 'CAMGLF', 'CAMGLG', 'CAMGLH', 'CAMHLA', 'CAMHLB', 'CAMHLC', 'CAMHLD', 'CAMHLE', 'CAMHLF', 'CAMHLG', 'CAMHLH', 'CBMALA', 'CBMALB', 'CBMALC', 'CBMALD', 'CBMALE', 'CBMALF', 'CBMALG', 'CBMALH', 'CBMBLA', 'CBMBLB', 'CBMBLC', 'CBMBLD', 'CBMBLE', 'CBMBLF', 'CBMBLG', 'CBMBLH', 'CBMCLA', 'CBMCLB', 'CBMCLC', 'CBMCLD', 'CBMCLE', 'CBMCLF', 'CBMCLG', 'CBMCLH', 'CBMDLA', 'CBMDLB', 'CBMDLC', 'CBMDLD', 'CBMDLE', 'CBMDLF', 'CBMDLG', 'CBMDLH', 'CBMELA', 'CBMELB', 'CBMELC', 'CBMELD', 'CBMELE', 'CBMELF', 'CBMELG', 'CBMELH', 'CBMFLA', 'CBMFLB', 'CBMFLC', 'CBMFLD', 'CBMFLE', 'CBMFLF', 'CBMFLG', 'CBMFLH', 'CBMGLA', 'CBMGLB', 'CBMGLC', 'CBMGLD', 'CBMGLE', 'CBMGLF', 'CBMGLG', 'CBMGLH', 'CBMHLA', 'CBMHLB', 'CBMHLC', 'CBMHLD', 'CBMHLE', 'CBMHLF', 'CBMHLG', 'CBMHLH', 'CCMALA', 'CCMALB', 'CCMALC', 'CCMALD', 'CCMALE', 'CCMALF', 'CCMALG', 'CCMALH', 'CCMBLA', 'CCMBLB', 'CCMBLC', 'CCMBLD', 'CCMBLE', 'CCMBLF', 'CCMBLG', 'CCMBLH', 'CCMCLA', 'CCMCLB', 'CCMCLC', 'CCMCLD', 'CCMCLE', 'CCMCLF', 'CCMCLG', 'CCMCLH', 'CCMDLA', 'CCMDLB', 'CCMDLC', 'CCMDLD', 'CCMDLE', 'CCMDLF', 'CCMDLG', 'CCMDLH', 'CCMELA', 'CCMELB', 'CCMELC', 'CCMELD', 'CCMELE', 'CCMELF', 'CCMELG', 'CCMELH', 'CCMFLA', 'CCMFLB', 'CCMFLC', 'CCMFLD', 'CCMFLE', 'CCMFLF', 'CCMFLG', 'CCMFLH', 'CCMGLA', 'CCMGLB', 'CCMGLC', 'CCMGLD', 'CCMGLE', 'CCMGLF', 'CCMGLG', 'CCMGLH', 'CCMHLA', 'CCMHLB', 'CCMHLC', 'CCMHLD', 'CCMHLE', 'CCMHLF', 'CCMHLG', 'CCMHLH', 'CDMALA', 'CDMALB', 'CDMALC', 'CDMALD', 'CDMALE', 'CDMALF', 'CDMALG', 'CDMALH', 'CDMBLA', 'CDMBLB', 'CDMBLC', 'CDMBLD', 'CDMBLE', 'CDMBLF', 'CDMBLG', 'CDMBLH', 'CDMCLA', 'CDMCLB', 'CDMCLC', 'CDMCLD', 'CDMCLE', 'CDMCLF', 'CDMCLG', 'CDMCLH', 'CDMDLA', 'CDMDLB', 'CDMDLC', 'CDMDLD', 'CDMDLE', 'CDMDLF', 'CDMDLG', 'CDMDLH', 'CDMELA', 'CDMELB', 'CDMELC', 'CDMELD', 'CDMELE', 'CDMELF', 'CDMELG', 'CDMELH', 'CDMFLA', 'CDMFLB', 'CDMFLC', 'CDMFLD', 'CDMFLE', 'CDMFLF', 'CDMFLG', 'CDMFLH', 'CDMGLA', 'CDMGLB', 'CDMGLC', 'CDMGLD', 'CDMGLE', 'CDMGLF', 'CDMGLG', 'CDMGLH', 'CDMHLA', 'CDMHLB', 'CDMHLC', 'CDMHLD', 'CDMHLE', 'CDMHLF', 'CDMHLG', 'CDMHLH', 'CEMALA', 'CEMALB', 'CEMALC', 'CEMALD', 'CEMALE', 'CEMALF', 'CEMALG', 'CEMALH', 'CEMBLA', 'CEMBLB', 'CEMBLC', 'CEMBLD', 'CEMBLE', 'CEMBLF', 'CEMBLG', 'CEMBLH', 'CEMCLA', 'CEMCLB', 'CEMCLC', 'CEMCLD', 'CEMCLE', 'CEMCLF', 'CEMCLG', 'CEMCLH', 'CEMDLA', 'CEMDLB', 'CEMDLC', 'CEMDLD', 'CEMDLE', 'CEMDLF', 'CEMDLG', 'CEMDLH', 'CEMELA', 'CEMELB', 'CEMELC', 'CEMELD', 'CEMELE', 'CEMELF', 'CEMELG', 'CEMELH', 'CEMFLA', 'CEMFLB', 'CEMFLC', 'CEMFLD', 'CEMFLE', 'CEMFLF', 'CEMFLG', 'CEMFLH', 'CEMGLA', 'CEMGLB', 'CEMGLC', 'CEMGLD', 'CEMGLE', 'CEMGLF', 'CEMGLG', 'CEMGLH', 'CEMHLA', 'CEMHLB', 'CEMHLC', 'CEMHLD', 'CEMHLE', 'CEMHLF', 'CEMHLG', 'CEMHLH', 'CFMALA', 'CFMALB', 'CFMALC', 'CFMALD', 'CFMALE', 'CFMALF', 'CFMALG', 'CFMALH', 'CFMBLA', 'CFMBLB', 'CFMBLC', 'CFMBLD', 'CFMBLE', 'CFMBLF', 'CFMBLG', 'CFMBLH', 'CFMCLA', 'CFMCLB', 'CFMCLC', 'CFMCLD', 'CFMCLE', 'CFMCLF', 'CFMCLG', 'CFMCLH', 'CFMDLA', 'CFMDLB', 'CFMDLC', 'CFMDLD', 'CFMDLE', 'CFMDLF', 'CFMDLG', 'CFMDLH', 'CFMELA', 'CFMELB', 'CFMELC', 'CFMELD', 'CFMELE', 'CFMELF', 'CFMELG', 'CFMELH', 'CFMFLA', 'CFMFLB', 'CFMFLC', 'CFMFLD', 'CFMFLE', 'CFMFLF', 'CFMFLG', 'CFMFLH', 'CFMGLA', 'CFMGLB', 'CFMGLC', 'CFMGLD', 'CFMGLE', 'CFMGLF', 'CFMGLG', 'CFMGLH', 'CFMHLA', 'CFMHLB', 'CFMHLC', 'CFMHLD', 'CFMHLE', 'CFMHLF', 'CFMHLG', 'CFMHLH', 'CGMALA', 'CGMALB', 'CGMALC', 'CGMALD', 'CGMALE', 'CGMALF', 'CGMALG', 'CGMALH', 'CGMBLA', 'CGMBLB', 'CGMBLC', 'CGMBLD', 'CGMBLE', 'CGMBLF', 'CGMBLG', 'CGMBLH', 'CGMCLA', 'CGMCLB', 'CGMCLC', 'CGMCLD', 'CGMCLE', 'CGMCLF', 'CGMCLG', 'CGMCLH', 'CGMDLA', 'CGMDLB', 'CGMDLC', 'CGMDLD', 'CGMDLE', 'CGMDLF', 'CGMDLG', 'CGMDLH', 'CGMELA', 'CGMELB', 'CGMELC', 'CGMELD', 'CGMELE', 'CGMELF', 'CGMELG', 'CGMELH', 'CGMFLA', 'CGMFLB', 'CGMFLC', 'CGMFLD', 'CGMFLE', 'CGMFLF', 'CGMFLG', 'CGMFLH', 'CGMGLA', 'CGMGLB', 'CGMGLC', 'CGMGLD', 'CGMGLE', 'CGMGLF', 'CGMGLG', 'CGMGLH', 'CGMHLA', 'CGMHLB', 'CGMHLC', 'CGMHLD', 'CGMHLE', 'CGMHLF', 'CGMHLG', 'CGMHLH', 'CHMALA', 'CHMALB', 'CHMALC', 'CHMALD', 'CHMALE', 'CHMALF', 'CHMALG', 'CHMALH', 'CHMBLA', 'CHMBLB', 'CHMBLC', 'CHMBLD', 'CHMBLE', 'CHMBLF', 'CHMBLG', 'CHMBLH', 'CHMCLA', 'CHMCLB', 'CHMCLC', 'CHMCLD', 'CHMCLE', 'CHMCLF', 'CHMCLG', 'CHMCLH', 'CHMDLA', 'CHMDLB', 'CHMDLC', 'CHMDLD', 'CHMDLE', 'CHMDLF', 'CHMDLG', 'CHMDLH', 'CHMELA', 'CHMELB', 'CHMELC', 'CHMELD', 'CHMELE', 'CHMELF', 'CHMELG', 'CHMELH', 'CHMFLA', 'CHMFLB', 'CHMFLC', 'CHMFLD', 'CHMFLE', 'CHMFLF', 'CHMFLG', 'CHMFLH', 'CHMGLA', 'CHMGLB', 'CHMGLC', 'CHMGLD', 'CHMGLE', 'CHMGLF', 'CHMGLG', 'CHMGLH', 'CHMHLA', 'CHMHLB', 'CHMHLC', 'CHMHLD', 'CHMHLE', 'CHMHLF', 'CHMHLG', 'CHMHLH']
diz={}
mm_array=calcola_media_mensile(df_dispositivo_emissioni)
mt_array=calcola_media_trimestrale(df_dispositivo_emissioni)
ms_array=calcola_media_semestrale(df_dispositivo_emissioni)

for pattern in pattern_tot:
    temp1 = pattern[pattern.index('C') + 1]
    temp2 = pattern[pattern.index('M') + 1]
    temp3 = pattern[pattern.index('L') + 1]

    diz[pattern]=[]
    for i in range(1,13):
        
        #ALGORITMO PER PRENDERE LE TRE MEDIE
        #MEDIA MENSILE
        num1=0
        if temp1=="A":
            num1=mm_array[i-1]/4
        elif temp1=="B":
            num1=mm_array[i-1]/2
        elif temp1=="C":
            num1=mm_array[i-1]*(3/4)
        elif temp1=="D":
            num1=mm_array[i-1]
        elif temp1=="E":
            num1=mm_array[i-1]*(5/4)
        elif temp1=="F":
            num1=mm_array[i-1]*(3/2)
        elif temp1=="G":
            num1=mm_array[i-1]*(7/4)
        elif temp1=="H":
            num1=mm_array[i-1]*2

        #MEDIA TRIMESTRALE
        num2=0
        trimestre=0
        
        #trimestrale
        if (i)<=3: trimestre=mt_array[0]
        elif (i)<=6: trimestre=mt_array[1]
        elif (i)<=9: trimestre=mt_array[2]
        else: trimestre=mt_array[3]

        if temp2=="A":
            num2=trimestre/4
        elif temp2=="B":
            num2=trimestre/2
        elif temp2=="C":
            num2=trimestre*(3/4)
        elif temp2=="D":
            num2=trimestre
        elif temp2=="E":
            num2=trimestre*(5/4)
        elif temp2=="F":
            num2=trimestre*(3/2)
        elif temp2=="G":
            num2=trimestre*(7/4)
        elif temp2=="H":
            num2=trimestre*2
        
        #MEDIA SEMESTRALE
        num3=0
        semestrale=0
        #semestrale
        if (i)<=6: semestrale=ms_array[0]
        else: semestrale=ms_array[1]

        if temp3=="A":
            num3=semestrale/4
        elif temp3=="B":
            num3=semestrale/2
        elif temp3=="C":
            num3=semestrale*(3/4)
        elif temp3=="D":
            num3=semestrale
        elif temp3=="E":
            num3=semestrale*(5/4)
        elif temp3=="F":
            num3=semestrale*(3/2)
        elif temp3=="G":
            num3=semestrale*(7/4)
        elif temp3=="H":
            num3=semestrale*2

        if num1<=num2 and num1<=num3:
            diz[pattern].append(num1)
        elif  num2<=num1 and num2<=num3:
            diz[pattern].append(num2)
        elif num3<=num1 and num3<=num2:
            diz[pattern].append(num3)

print(mm_array)
print(mt_array)
print(ms_array)
print(diz)
    

[0.0, 5361.964285714285, 6614.870967741936, 8858.8, 7666.451612903225, 13478.5, 6155.0, 2841.5806451612902, 7151.266666666666, 10217.741935483871, 0.0, 0.0]
[3946.6222222222223, 9975.593406593407, 5363.391304347826, 3442.9347826086955]
[6977.762430939227, 4403.163043478261]
{'CAMALA': [0.0, 986.6555555555556, 986.6555555555556, 1744.4406077348067, 1744.4406077348067, 1744.4406077348067, 1100.7907608695652, 710.3951612903226, 1100.7907608695652, 860.7336956521739, 0.0, 0.0], 'CAMALB': [0.0, 986.6555555555556, 986.6555555555556, 2214.7, 1916.6129032258063, 2493.8983516483518, 1340.8478260869565, 710.3951612903226, 1340.8478260869565, 860.7336956521739, 0.0, 0.0], 'CAMALC': [0.0, 986.6555555555556, 986.6555555555556, 2214.7, 1916.6129032258063, 2493.8983516483518, 1340.8478260869565, 710.3951612903226, 1340.8478260869565, 860.7336956521739, 0.0, 0.0], 'CAMALD': [0.0, 986.6555555555556, 986.6555555555556, 2214.7, 1916.6129032258063, 2493.8983516483518, 1340.8478260869565, 710.3951612903226